## Introdução

Estaremos usando dados de clientes de um [distribuidor atacadista português](https://archive.ics.uci.edu/ml/datasets/Wholesale+customers?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel- SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) para clustering. Este arquivo de dados é chamado `Wholesale_Customers_Data`.

Ele contém os seguintes recursos:


* Fresco: gasto anual (m.u.) em produtos frescos
* Leite: gasto anual (m.u.) com produtos lácteos
* Mercearia: gasto anual (m.u.) em produtos de mercearia
* Congelados: gasto anual (m.u.) com produtos congelados
* Detergentes_Papel: gasto anual (m.u.) em detergentes e produtos de papel
* Delicatessen: gasto anual (m.u.) em produtos de delicatessen
* Canal: canal cliente (1: hotel/restaurante/café ou 2: retalho)
* Região: região do cliente (1: Lisboa, 2: Porto, 3: Outro)

Nesses dados, os valores de todos os gastos são dados em uma unidade arbitrária (m.u. = unidade monetária).

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import seaborn as sns, pandas as pd, numpy as np

In [ ]:
import os, pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt


## Parte 1

Nesta seção, iremos:

* Importe os dados e verifique os tipos de dados.
* Elimine as colunas de canal e região, pois elas não serão usadas, pois nos concentramos em colunas numéricas para este exemplo.
* Converta as colunas restantes em floats, se necessário.
* Copie esta versão dos dados (usando o método `copy`) para uma variável para preservá-la. Nós o usaremos mais tarde.

In [ ]:
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/Wholesale_Customers_Data.csv', sep=',')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data = data.drop(['Channel', 'Region'], axis=1)

In [ ]:
data.dtypes

In [ ]:
# converte os floats
for col in data.columns:
    data[col] = data[col].astype(np.float)

Preservo o dataset original


In [ ]:
data_orig = data.copy()

## Parte 2

Assim como na lição anterior, precisamos garantir que os dados sejam dimensionados e (relativamente) distribuídos normalmente.

* Examine a correlação e a inclinação.
* Execute quaisquer transformações e dimensione dados usando seu método de dimensionamento favorito.
* Veja os gráficos de correlação pairwise dos novos dados.

In [ ]:
corr_mat = data.corr()

for x in range(corr_mat.shape[0]):
    corr_mat.iloc[x,x] = 0.0
    
corr_mat


Como antes, as duas categorias com suas respectivas variáveis ​​mais fortemente correlacionadas.


In [ ]:
corr_mat.abs().idxmax()

Examine os valores de inclinação e a transformação de log. Parece que todos precisam.


In [ ]:
log_columns = data.skew().sort_values(ascending=False)
log_columns = log_columns.loc[log_columns > 0.75]

log_columns

In [ ]:
for col in log_columns.index:
    data[col] = np.log1p(data[col])


Escale os dados novamente. Vamos usar `MinMaxScaler` desta vez apenas para misturar as coisas.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

for col in data.columns:
    data[col] = mms.fit_transform(data[[col]]).squeeze()


Visualize a relação entre as variáveis.

In [ ]:
sns.set_context('notebook')
sns.set_style('white')
sns.pairplot(data);

## Parte 3

Nesta seção, iremos:
* Usando a [função de pipeline] do Scikit-learn (http://scikit-learn.org/stable/modules/pipeline.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187 ENSkillsNetwork821-2023-01- 01), recrie o esquema de pré-processamento de dados acima (transformação e dimensionamento) usando um pipeline. Se você usou uma função de aprendizado não Scikit para transformar os dados (por exemplo, a função de log do NumPy), verifique a classe de transformador personalizada chamada [`FunctionTransformer`](http://scikit-learn.org/stable/modules/preprocessing.html? utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01#custom-transformers).
* Use o pipeline para transformar os dados originais que foram armazenados no final da pergunta 1.
* Compare os resultados com os dados originais para verificar se tudo funcionou.

*Observação:* Scikit-learn tem uma função `Pipeline` mais flexível e uma versão de atalho chamada `make_pipeline`. Qualquer um pode ser usado. Além disso, se diferentes transformações precisam ser executadas nos dados, um [`FeatureUnion`](http://scikit-learn.org/stable/modules/pipeline.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA- SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01#featureunion-composite-feature-spaces) podem ser usados.


In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

# O transformador de log NumPy personalizado
log_transformer = FunctionTransformer(np.log1p)

# O pipeline
estimators = [('log1p', log_transformer), ('minmaxscale', MinMaxScaler())]
pipeline = Pipeline(estimators)

# Converter os dados originais
data_pipe = pipeline.fit_transform(data_orig)

In [ ]:
np.allclose(data_pipe, data)

## Parte 4

Nesta seção, iremos:
* Execute o PCA com `n_components` variando de 1 a 5.
* Armazene a quantidade de variação explicada para cada número de dimensões.
* Também armazene a importância do recurso para cada número de dimensões. *Dica:* O PCA não fornece isso explicitamente depois que um modelo é ajustado, mas as propriedades `components_` podem ser usadas para determinar algo que se aproxime da importância. Como você decidiu fazer isso depende inteiramente de você.
* Plote a variância explicada e as importâncias dos recursos.

In [ ]:
from sklearn.decomposition import PCA

pca_list = list()
feature_weight_list = list()

# Adapta-se a uma variedade de modelos de PCA

for n in range(1, 6):
    
    # Criar e ajustar o modelo
    PCAmod = PCA(n_components=n)
    PCAmod.fit(data)
    
    # Armazene o modelo e a variância
    pca_list.append(pd.Series({'n':n, 'model':PCAmod,
                               'var': PCAmod.explained_variance_ratio_.sum()}))
    
    # Calcule e armazene as importâncias dos recursos
    abs_feature_values = np.abs(PCAmod.components_).sum(axis=0)
    feature_weight_list.append(pd.DataFrame({'n':n, 
                                             'features': data.columns,
                                             'values':abs_feature_values/abs_feature_values.sum()}))
    
pca_df = pd.concat(pca_list, axis=1).T.set_index('n')
pca_df

In [ ]:
features_df = (pd.concat(feature_weight_list)
               .pivot(index='n', columns='features', values='values'))

features_df

In [ ]:
sns.set_context('talk')
ax = pca_df['var'].plot(kind='bar')

ax.set(xlabel='Number of dimensions',
       ylabel='Percent explained variance',
       title='Explained Variance vs Dimensions');


E aqui está um gráfico de importâncias de recursos.


In [ ]:
ax = features_df.plot(kind='bar', figsize=(13,8))
ax.legend(loc='upper right')
ax.set(xlabel='Number of dimensions',
       ylabel='Relative importance',
       title='Feature importance vs Dimensions');

## Parte 5

Nesta seção, iremos:
* Ajuste um modelo `KernelPCA` com `kernel='rbf'`. Você pode escolher quantos componentes e quais valores usar para os outros parâmetros (`rbf` refere-se a um kernel de função de base radial, e o parâmetro `gamma` rege a escala desse kernel e normalmente varia entre 0 e 1). Vários outros [kernels](https://scikit-learn.org/stable/modules/metrics.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821- 2023-01-01) pode ser Tentei e até passou os parâmetros de validação cruzada da SS (consulte este [exemplo] (https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html?utm_exinfluencher&soarch_sourcegits. 555 & utm_id = na-skillsnetwork- Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01)).
* Se você quiser mexer um pouco mais, use `GridSearchCV` para ajustar os parâmetros do modelo `KernelPCA`.

A segunda etapa é complicada, pois as pesquisas em grade geralmente são usadas para métodos de aprendizado de máquina supervisionado e dependem de métricas de pontuação, como precisão, para determinar o melhor modelo. No entanto, uma função de pontuação personalizada pode ser escrita para `GridSearchCV`, onde maior é melhor para o resultado da função de pontuação.

O que essa métrica envolveria para o PCA? E a porcentagem da variância explicada? Ou talvez o erro quadrático médio negativo nos dados depois de terem sido transformados e depois transformados inversamente?

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Pontuador personalizado--use o rmse negativo da transformação inversa
def scorer(pcamodel, X, y=None):

    try:
        X_val = X.values
    except:
        X_val = X
        
    # Calcular e transformar inversamente os dados
    data_inv = pcamodel.fit(X_val).transform(X_val)
    data_inv = pcamodel.inverse_transform(data_inv)
    
    # O cálculo do erro
    mse = mean_squared_error(data_inv.ravel(), X_val.ravel())
    
    # Valores maiores são melhores para pontuadores, então tome valor negativo
    return -1.0 * mse

# Os parâmetros de pesquisa da grade
param_grid = {'gamma':[0.001, 0.01, 0.05, 0.1, 0.5, 1.0],
              'n_components': [2, 3, 4]}

# A grade de pesquisa
kernelPCA = GridSearchCV(KernelPCA(kernel='rbf', fit_inverse_transform=True),
                         param_grid=param_grid,
                         scoring=scorer,
                         n_jobs=-1)


kernelPCA = kernelPCA.fit(data)

kernelPCA.best_estimator_

## Parte 6

Vamos explorar como a precisão de nosso modelo pode mudar se incluirmos um `PCA` em nosso pipeline de construção de modelo. Vamos planejar usar a classe `Pipeline` do sklearn e criar um pipeline que tenha as seguintes etapas:
<ol>
  <li>Um escalador</li>
  <li>`PCA(n_components=n)`</li>
  <li>`Regressão Logística`</li>
</ol>

* Carregue os dados de Atividade Humana dos conjuntos de dados.
* Escreva uma função que receba um valor de `n` e faça o pipeline acima, depois preveja a coluna "Atividade" em um StratifiedShuffleSplit 5 vezes e retorne a precisão média do teste
* Para vários valores de n, chame a função acima e armazene as precisões médias.
* Plote a precisão média por número de dimensões.


In [ ]:
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/Human_Activity_Recognition_Using_Smartphones_Data.csv', sep=',')

In [ ]:
data.columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = data.drop('Activity', axis=1)
y = data.Activity
sss = StratifiedShuffleSplit(n_splits=5, random_state=42)

def get_avg_score(n):
    pipe = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n)),
        ('estimator', LogisticRegression(solver='liblinear'))
    ]
    pipe = Pipeline(pipe)
    scores = []
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        pipe.fit(X_train, y_train)
        scores.append(accuracy_score(y_test, pipe.predict(X_test)))
    return np.mean(scores)


ns = [10, 20, 50, 100, 150, 200, 300, 400]
score_list = [get_avg_score(n) for n in ns]

In [ ]:
sns.set_context('talk')

ax = plt.axes()
ax.plot(ns, score_list)
ax.set(xlabel='Number of Dimensions',
       ylabel='Average Accuracy',
       title='LogisticRegression Accuracy vs Number of dimensions on the Human Activity Dataset')
ax.grid(True)